In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [2]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
196052,CA,F,2011,Reet,9
134321,CA,F,1995,Thanya,6
33114,CA,F,1954,Lorene,44
207158,CA,F,2014,Aiza,17
146568,CA,F,1999,Kyara,25


## Goal 1: Find the most popular baby name in California in 2018

In [8]:
babynames.query('Year == 2018').sort_values("Count", ascending=False)

,State,Sex,Year,Name,Count
221210,CA,F,2018,Emma,2752
396810,CA,M,2018,Noah,2574
221211,CA,F,2018,Mia,2506
221212,CA,F,2018,Olivia,2474
396811,CA,M,2018,Liam,2415
...,...,...,...,...,...
224754,CA,F,2018,Marni,5
224755,CA,F,2018,Marnie,5
224756,CA,F,2018,Masa,5
224757,CA,F,2018,Matilde,5


## Goal 2: Baby names that start with j. 

In [16]:
babynames[babynames["Name"].str.startswith("J")]["Name"].unique()

array(['Josephine', 'Jean', 'Jessie', ..., 'Josiel', 'Jayco', 'Jaivin'],
      dtype=object)

## Goal 3: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [27]:
babynames["Name Length"] = babynames["Name"].map(len)
babynames_by_length = babynames.sort_values(by = "Name Length")
babynames_by_length = babynames_by_length.drop("Name Length", axis=1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
83016,CA,F,1979,Ji,5
331174,CA,M,1993,Vu,5
298821,CA,M,1978,Al,13
277555,CA,M,1962,Ty,55
404824,CA,M,2020,Jj,6


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [28]:
babynames["Name Length"] = babynames["Name"].str.len()
babynames_by_length = babynames.sort_values(by = "Name Length")
babynames_by_length = babynames_by_length.drop("Name Length", axis=1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
83016,CA,F,1979,Ji,5
331174,CA,M,1993,Vu,5
298821,CA,M,1978,Al,13
277555,CA,M,1962,Ty,55
404824,CA,M,2020,Jj,6


Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [17]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count
83016,CA,F,1979,Ji,5
331174,CA,M,1993,Vu,5
298821,CA,M,1978,Al,13
277555,CA,M,1962,Ty,55
404824,CA,M,2020,Jj,6


How does this work exactly? Let's break it into pieces.

In [18]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

83016     2
331174    2
298821    2
277555    2
404824    2
Name: Name, dtype: int64

In [19]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Index([ 83016, 331174, 298821, 277555, 404824,  35741, 242808, 363478, 130111,
       380198,
       ...
       343125, 341130, 331124, 340954, 320534, 337819, 325562, 316193, 317627,
       102512],
      dtype='int64', length=413894)

In [20]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count
83016,CA,F,1979,Ji,5
331174,CA,M,1993,Vu,5
298821,CA,M,1978,Al,13
277555,CA,M,1962,Ty,55
404824,CA,M,2020,Jj,6
